In [1]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ['KERAS_BACKEND']='tensorflow' # Why theano why not
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline

Using TensorFlow backend.


In [2]:
from keras.callbacks import callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

ImportError: cannot import name 'callback'

#### Function to calculate f1 score

In [16]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
'''model.compile(loss='binary_crossentropy',
          optimizer= "adam",
          metrics=[f1])'''

'model.compile(loss=\'binary_crossentropy\',\n          optimizer= "adam",\n          metrics=[f1])'

In [4]:
df = pd.read_csv('/home/mirafra/Desktop/nlp_project/english/agr_en_train.csv',header = None)
df.head(4)

,0,1,2
0,facebook_corpus_msr_1723796,Well said sonu..you have courage to stand agai...,1
1,facebook_corpus_msr_466073,"Most of Private Banks ATM's Like HDFC, ICICI e...",0
2,facebook_corpus_msr_1493901,"Now question is, Pakistan will adhere to this?",1
3,facebook_corpus_msr_405512,Pakistan is comprised of fake muslims who does...,1


In [5]:
dataset = df.loc[:,1:2]
dataset.columns = ['message', 'class']
print(dataset)

                                                 message  class
0      Well said sonu..you have courage to stand agai...      1
1      Most of Private Banks ATM's Like HDFC, ICICI e...      0
2         Now question is, Pakistan will adhere to this?      1
3      Pakistan is comprised of fake muslims who does...      1
4      ??we r against cow slaughter,so of course it w...      0
5      Wondering why Educated Ambassador is strugglin...      0
6      How does inflation react to all the after shoc...      0
7      Not good job.....this guis creating a problem ...      0
8      This is a false news Indian media is simply mi...      0
9      no permanent foes, no permanent friends. inter...      0
10     Deepak Kumar Sharma Saab...chalo aap ki Ye baa...      0
11     Communist parties killed lacks of opponents in...      1
12     Why you guys counter the modi govt decisions, ...      0
13              Rss is 3 time ban terrorist organization      1
14     No \nSame acting ll be there .. \

In [7]:
def clean_str(string):
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

In [6]:
def clean_str(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [7]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [8]:
# reading data
#df = pd.read_excel('dataset.xlsx')
data = dataset.dropna()
data = data.reset_index(drop=True)
print('Shape of dataset ',data.shape)
print(data.columns)
print('No. of unique classes',len(set(data['class'])))
countA=0
countB=0
countC=0
for item in data['class']:
    if item == 1:
        countA +=1
    elif item == 0:
        countB +=1
    elif item == 'CAG':
        countC +=1
print("NAG: ", countA)
print("OAG: ",countB)
print("CAG: ",countC)

Shape of dataset  (11999, 2)
Index(['message', 'class'], dtype='object')
No. of unique classes 2
NAG:  2708
OAG:  9291
CAG:  0


In [9]:
macronum=sorted(set(data['class']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

data['class']=data['class'].apply(fun)

In [10]:
texts = []
labels = []


for idx in range(data.message.shape[0]):
    text = BeautifulSoup(data.message[idx])
    texts.append(clean_str(str(text.get_text().encode())))

for idx in data['class']:
    labels.append(idx)

#### Tokennization

In [11]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Number of Unique Tokens',len(word_index))

Number of Unique Tokens 26361


#### Lemmatization

In [ ]:
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [12]:
datas = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of Data Tensor:', datas.shape)
print('Shape of Label Tensor:', labels.shape)

indices = np.arange(datas.shape[0])
np.random.shuffle(indices)
datas = datas[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = datas[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = datas[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Shape of Data Tensor: (11999, 1000)
Shape of Label Tensor: (11999, 2)


In [13]:
embeddings_index = {}
f = open('glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [14]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [17]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(len(macronum), activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[f1])

print("Simplified convolutional neural network")
model.summary()
cp=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)

Simplified convolutional neural network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         2636200   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 

In [ ]:
history=model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=2,callbacks=[cp])

Train on 9600 samples, validate on 2399 samples
Epoch 1/15
9600/9600 [==============================] - 668s 70ms/step - loss: 3.5918 - f1: 0.7759 - val_loss: 3.7219 - val_f1: 0.7678
Epoch 2/15
9600/9600 [==============================] - 645s 67ms/step - loss: 3.5918 - f1: 0.7759 - val_loss: 3.7219 - val_f1: 0.7678
Epoch 3/15
9600/9600 [==============================] - 652s 68ms/step - loss: 3.5918 - f1: 0.7759 - val_loss: 3.7219 - val_f1: 0.7678
Epoch 4/15
9600/9600 [==============================] - 621s 65ms/step - loss: 3.5918 - f1: 0.7759 - val_loss: 3.7219 - val_f1: 0.7678
Epoch 5/15
9600/9600 [==============================] - 649s 68ms/step - loss: 3.5918 - f1: 0.7759 - val_loss: 3.7219 - val_f1: 0.7678
Epoch 6/15
9600/9600 [==============================] - 641s 67ms/step - loss: 3.5918 - f1: 0.7759 - val_loss: 3.7219 - val_f1: 0.7678
Epoch 7/15
9600/9600 [==============================] - 645s 67ms/step - loss: 3.5918 - f1: 0.7759 - val_loss: 3.7219 - val_f1: 0.7678
Epoch 8

In [18]:
history=model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=2, batch_size=2,callbacks=[cp])

Train on 9600 samples, validate on 2399 samples
Epoch 1/2
9600/9600 [==============================] - 652s 68ms/step - loss: 3.5905 - f1: 0.7759 - val_loss: 3.7219 - val_f1: 0.7678
Epoch 2/2
   2/9600 [..............................] - ETA: 10:25 - loss: 1.0960e-07 - f1: 1.0000

/home/mirafra/anaconda3/envs/abhishek/lib/python3.6/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


9600/9600 [==============================] - 623s 65ms/step - loss: 3.5918 - f1: 0.7759 - val_loss: 3.7219 - val_f1: 0.7678


### Save the trained model

In [19]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk


### using the saved model for prediction

In [1]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#score = loaded_model.evaluate(x_val, y_val, verbose=0)
#print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Using TensorFlow backend.


Loaded model from disk


#### Make prediction using trained model

In [ ]:
pred_val = loaded_model.predict()
labels = np.argmax(probas, axis=-1)

#### Label encoder

In [18]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(["NAG","OAG","CAG"])
print(list(le.classes_))
print(le.transform(["OAG", "OAG", "NAG","CAG"]))
print(list(le.inverse_transform([2, 2, 1,0])))

['CAG', 'NAG', 'OAG']
[2 2 1 0]
['OAG', 'OAG', 'NAG', 'CAG']


### Make prediction using saved model

In [34]:
from sklearn.datasets.samples_generator import make_blobs
pred = loaded_model.predict(datas_new)
#accuracy_score(y_val, pred)
labels = np.argmax(pred, axis=-1)
print(pred)
print(labels)

[[0.75133294 0.24866706]
 [0.8484862  0.15151377]
 [0.49108267 0.5089173 ]
 ...
 [0.92430913 0.07569086]
 [0.5553795  0.4446204 ]
 [0.80169934 0.19830066]]
[0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0
 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1
 0 0 0 1 0 0 0 0 0 

In [18]:
import pandas as pd
#df_test = pd.read_csv('/home/mirafra/Desktop/nlp_project/sample_output/Test A Release/testset-taska.csv')
df_test = pd.read_csv('/home/mirafra/Desktop/nlp_project/english/agr_en_test.csv',delimiter='\t',encoding='utf-8')
x_val = df_test['tweet']
print(x_val)

0      #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
1      #ConstitutionDay is revered by Conservatives, ...
2      #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
3      #Watching #Boomer getting the news that she is...
4      #NoPasaran: Unity demo to oppose the far-right...
5               . . . What the fuck did he do this time?
6      #RAP is a form of ART! Used to express yoursel...
7      @USER Do you get the feeling he is kissing @US...
8      5 Tips to Enhance Audience Connection on Faceb...
9      #BiggBossTamil janani won the task. She is goi...
10     #Conservatives - the party of low taxation 😂 #...
11     𝓒-𝓒𝓮𝓭𝓻𝓪𝓷..  The Nord cannot make a single move...
12       #ConsTOO THE PLACE FOR FED UP CONSERVATIVES !!!
13     #GreatAwakening #QAnon #PatriotsUnited #WWG1WG...
14                           @USER Nigga ware da hits at
15     #StopKavanaugh he is liar like the rest of the...
16     #Dayspromo week of September 17th #days #dool ...
17     #BiggBossTelugu2 Let’s a

In [29]:
df_test_new = df_test.loc[:,1:2]
df_test_new.columns = ['message', 'class']
print(df_test_new)

                                                message class
0     The quality of re made now makes me think it i...   CAG
1     @siva \nHow is ur mother???\nHow is ur wife???...   NAG
2     Also see ....hw ur RSS activist caught in Burk...   NAG
3     On the death of 2 jawans in LOC CROSS FIRING\n...   NAG
4     Modi ho ya Manmohan singh saala yeh log kuch n...   OAG
5     No discrimination in policies and NON protecti...   CAG
6     As Mr Ashwini gujral said once nifty will touc...   NAG
7     After seeing your comment guys.. I think sonu ...   CAG
8     What has so far Mr.Yechuri done for this Count...   CAG
9     SONIA SHENOY.....A KONKONI.....WRONGLY INTERPR...   CAG
10              Now all AAPtards will abuse ANNA HAJARE   NAG
11    I salute ..@ Neel Patel,,U r just amazing. Eac...   NAG
12    Hi Sonia!! Markets & reliance zooming like roc...   NAG
13    These kashmiri youth are radicalized jihadi te...   OAG
14    If you want to construct a smooth road you hav...   CAG
15    Ph

In [19]:
def clean_str(string):
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

In [20]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [22]:
# reading data
#df = pd.read_excel('dataset.xlsx')
df_test_new = x_val.dropna()
df_test_new = df_test_new.reset_index(drop=True)
print('Shape of dataset ',df_test_new.shape)
#print(df_test_new.columns)
#print('No. of unique classes',len(set(df_test_new['class'])))
countA=0
countB=0
countC=0
'''for item in df_test_new['class']:
    if item == 'NAG':
        countA +=1
    elif item == 'OAG':
        countB +=1
    elif item == 'CAG':
        countC +=1
print("NAG: ", countA)
print("OAG: ",countB)
print("CAG: ",countC)'''

Shape of dataset  (860,)


'for item in df_test_new[\'class\']:\n    if item == \'NAG\':\n        countA +=1\n    elif item == \'OAG\':\n        countB +=1\n    elif item == \'CAG\':\n        countC +=1\nprint("NAG: ", countA)\nprint("OAG: ",countB)\nprint("CAG: ",countC)'

In [7]:
macronum=sorted(set(df_test_new['class']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

df_test_new['class']=df_test_new['class'].apply(fun)

In [28]:
texts_new = []
labels_new = []


for idx in range(df_test_new.shape[0]):
    text = BeautifulSoup(df_test_new[idx])
    texts_new.append(clean_str(str(text.get_text().encode())))

#for idx in df_test_new['class']:
#    labels_new.append(idx)

In [29]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts_new)
sequences = tokenizer.texts_to_sequences(texts_new)

word_index = tokenizer.word_index
print('Number of Unique Tokens',len(word_index))

Number of Unique Tokens 5463


In [30]:
datas_new = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels_new = to_categorical(np.asarray(labels_new))
print('Shape of Data Tensor:', datas_new.shape)
#print('Shape of Label Tensor:', labels_new.shape)

indices = np.arange(datas_new.shape[0])
np.random.shuffle(indices)
datas_new = datas_new[indices]
#labels_new = labels_new[indices]

Shape of Data Tensor: (860, 1000)


#### Glove for word embedding

In [ ]:
embeddings_index = {}
f = open('glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

In [32]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [35]:
from sklearn.datasets.samples_generator import make_blobs
pred = loaded_model.predict(datas_new)
#accuracy_score(y_val, pred)
labels = np.argmax(pred, axis=-1)
print(pred)
print(labels)

[[0.75133294 0.24866706]
 [0.8484862  0.15151377]
 [0.49108267 0.5089173 ]
 ...
 [0.92430913 0.07569086]
 [0.5553795  0.4446204 ]
 [0.80169934 0.19830066]]
[0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0
 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1
 0 0 0 1 0 0 0 0 0 

In [37]:
labels[2]

1

In [42]:
Test_Result = {'id':df_test['id'],'labels':labels}

In [51]:
df = pd.DataFrame(Test_Result)

In [52]:
df

,id,labels
0,15923,0
1,27014,0
2,30530,1
3,13876,0
4,60133,1
5,83681,0
6,96874,0
7,65507,0
8,78910,0
9,46363,1


In [47]:
df.to_csv('result_task_A')